Vision and Cognition System - Project

In [18]:
import pandas as pd
import numpy as np

The main link and paper that we need to follow is [this](https://github.com/TIBHannover/GeoEstimation)

In [15]:


url = pd.read_csv(r"C:\Users\latta\GitHub\Vision_Project\mp16_urls.csv", names = ["name","url"])
#read the pandas csv requires 20 sec

Maybe we will need this [site](https://opensourceoptions.com/blog/use-python-to-download-multiple-files-or-urls-in-parallel/)

Seguendo il paper [Where in the World is this Image? Transformer-based Geo-localization in the Wild](file:///C:/Users/latta/Downloads/Translocator_with_supplementary.pdf) tra le reference c'è il dataset del paper [Revisiting IM2GPS in the Deep Learning Era](https://arxiv.org/abs/1705.04838) che dà accesso al [sito](https://paperswithcode.com/dataset/yfcc100m) da cui si arriva in [Multimedia Commons](http://www.multimediacommons.org/). Qui comunque ci si rende conta che serve avere qualche tipo di account con i services di amazon per poter usare il dataset.

In [19]:
print(np.shape(url))

(4723695, 2)


In [26]:
display(url['url'][10])
display(url['name'][10])

'http://farm1.staticflickr.com/91/236416609_d7daddff3b.jpg'

'f2/52/236416609.jpg'

Davide ha trovato questo che forse è meglio [kaggle](https://www.kaggle.com/code/habedi/inspect-the-dataset/data)

Qui ci sono dei links che potrebbero essere usati con colab col comando [!wget](https://qualinet.github.io/databases/image/world_wide_scale_geotagged_image_dataset_for_automatic_image_annotation_and_reverse_geotagging/)